In [160]:
%pip install crewai langchain_openai langchain_community python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [161]:

from bs4 import BeautifulSoup
from typing import List



In [162]:

from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI
from langchain.tools import tool
import re
from langchain_community.document_loaders import PyMuPDFLoader
import requests


In [163]:
amora_books_categories = [
    {
        "name": "Fiction Favorites",
        "url": "https://amorabooks.com/category/fiction-favorites"
    },
    {
        "name": "Non-Fiction Narratives",
        "url": "https://amorabooks.com/category/non-fiction-narratives"
    },
    {
        "name": "Mystery & Suspense",
        "url": "https://amorabooks.com/category/mystery-suspense"
    },
    {
        "name": "Sci-Fi & Fantasy Realms",
        "url": "https://amorabooks.com/category/scifi-fantasy-realms"
    }]

title = "explore your new online library"

text = """Explore Our Book Selection at Amora Books
Welcome to Amora Books, your gateway to a world of literary wonders! Whether you’re a passionate reader, an occasional browser, or just beginning your literary journey, we have something for everyone. Dive into our vast and diverse collection of books that span genres, authors, and interests from around the globe.

Fiction
Lose yourself in the enchanting realms of fiction. From heartwarming romance to edge-of-your-seat thrillers, our fiction selection has something for every taste. Explore classic literature, contemporary novels, fantasy adventures, and science fiction epics that transport you to new worlds. Delve into narratives crafted by master storytellers and discover new voices that resonate with your soul.

Non-Fiction
Satisfy your curiosity and expand your knowledge with our extensive range of non-fiction titles. Whether you’re interested in history, science, biographies, or self-help, our collection offers insights and inspiration. Learn from experts, challenge your perspectives, and gain a deeper understanding of the world around you.

Children's Books
Inspire the young minds in your life with our delightful collection of children’s books. From timeless fairy tales to educational stories, our books cater to all ages and interests. Ignite their imagination with captivating illustrations and engaging tales that will foster a lifelong love of reading.

Young Adult
Discover the vibrant world of young adult literature that speaks to the heart of teenage experiences. Our selection includes contemporary tales, dystopian adventures, and fantasy sagas that capture the essence of youth. Explore stories of self-discovery, friendship, and resilience that resonate with readers of all ages.

Mystery & Thriller
Uncover the secrets and twists hidden within our mystery and thriller collection. From classic whodunits to psychological thrillers, these page-turners will keep you guessing until the very end. Experience the thrill of suspense and unravel the enigmas crafted by the best in the genre.

Romance
Indulge in tales of love and passion with our captivating romance novels. Whether you prefer historical romance, contemporary love stories, or paranormal pairings, our selection will sweep you off your feet. Discover stories that explore the beauty and complexity of human relationships.

Science Fiction & Fantasy
Venture into realms of wonder with our science fiction and fantasy books. From epic quests to futuristic societies, explore worlds beyond your imagination. Our collection includes beloved classics and innovative new works that push the boundaries of creativity.

Staff Picks
Not sure where to start? Check out our Staff Picks section for recommendations from our knowledgeable team. These hand-picked selections showcase some of our favorite reads and hidden gems that we believe you’ll love."""


In [164]:

@tool
def get_webpage_category_pages(url: str):
    """
    Reads the webpage with a given URL and returns the category pages.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for HTTP errors
        return response.text
    except requests.exceptions.RequestException as e:
        return str(e)


In [165]:
# Load your OPENAI_API_KEY from your .env file
load_dotenv()
# The model for the agents
model = ChatOpenAI(model_name="gpt-4o", temperature=0.8)
original_url = "http://ezshop.ca"

# Agents



# agent_semantic_category_analyzer = Agent(
#     role='Semantic Category Analyzer',
#     goal='Identify semantically relevant categories',
#     backstory="""You are responsible for analyzing all categories on the live website 
#     and finding semantically relevant categories to the one we are writing about. 
#     You need to return four categories that are semantically related to the category 
#     as well as the links for each category.""",
#     tools=[get_category_urls],  # Add the updated custom scraping tool
#     llm=model,
#     max_iter=15,
#     max_execution_time=60,
#     verbose=True,
#     allow_delegation=True,
#     cache=True
# )

agent_ecommerce_content_creator = Agent(
    role='E-commerce Content Creator',
    goal='Write comprehensive text for the category',
    backstory="""You are responsible for receiving information about the website 
    and the category we are writing the description for. You then write a comprehensive 
    and engaging text for the category.""",
    llm=model,
    max_iter=15,
     max_execution_time=60,
    verbose=True,
    allow_delegation=True,
    cache=True
)

agent_link_integration_specialist = Agent(
    role='Link Integration Specialist',
    goal='Create unique interlinking sentences',
    backstory="""You are responsible for receiving the category links and text, 
    creating four unique interlinking sentences. Each sentence must have one 
    semantically related category name linked to its respective URL provided 
    by the Semantic Category Analyzer. You must avoid redundancy and overlap.""",

    llm=model,
    max_iter=15,
    max_execution_time=60,
    verbose=True,
    allow_delegation=True,
    cache=True
)

agent_content_integration_manager = Agent(
    role='Content Integration Manager',
    goal='Combine text and interlinking sentences into a cohesive description',
    backstory="""You are responsible for receiving the category text from 
    the E-commerce Content Creator and the interlinking sentences from the 
    Link Integration Specialist. You need to combine them into a cohesive 
    and engaging category description.""",
    llm=model,
    max_iter=15,
    max_execution_time=60,
    verbose=True,
    allow_delegation=True,
    cache=True
)


# Tasks


In [166]:


# Task for analyzing categories
# task_analyze_categories = Task(
#     description="""Analyze categories on the live website and identify semantically relevant ones. url = {url}""",
#     expected_output='List of four categories that are semantically related along with their links.',
#     agent=agent_semantic_category_analyzer,  # Ensure this is correctly defined in your system
#     tools=[get_category_urls],  # Use the custom scraping tool to fetch category URLs
#     input_schema={'url': str},  # Define the expected input schema, e.g., a URL of the website
#     output_schema=List[dict]    # Define the expected output schema
# )

# Task for writing category text
task_write_category_text = Task(
    description='Write a comprehensive text for the category {title}',
    expected_output='A well-written category text',
    agent=agent_ecommerce_content_creator
    # No tools specified, assuming text generation is done by LLM
)

# Task for creating interlinking sentences
task_create_interlinking_sentences = Task(
    description='Create four unique interlinking sentences for the category text {input}',
    expected_output='Four interlinking sentences that fit the category text',
    agent=agent_link_integration_specialist,
)

# Task for integrating content
task_integrate_content = Task(
    description='Combine the category text and interlinking sentences into a cohesive description for the text {text}',
    expected_output='A perfect category description',
    agent=agent_content_integration_manager
    # No tools specified, assuming integration is done by LLM
)


In [167]:
crew = Crew(
    agents=[
        agent_link_integration_specialist,
        agent_content_integration_manager
        

    ],
    tasks=[
        task_create_interlinking_sentences,
        task_integrate_content
   
    ],
    verbose=True
)

# Start the crew's execution with the URL input for category analysis
result = crew.kickoff(inputs={"input": amora_books_categories, "text": text})
print(result)

2024-08-06 17:13:12,813 - 8507938048 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


 [2024-08-06 17:13:12][DEBUG]: == Working Agent: Link Integration Specialist
 [2024-08-06 17:13:12][INFO]: == Starting Task: Create four unique interlinking sentences for the category text [{'name': 'Fiction Favorites', 'url': 'https://amorabooks.com/category/fiction-favorites'}, {'name': 'Non-Fiction Narratives', 'url': 'https://amorabooks.com/category/non-fiction-narratives'}, {'name': 'Mystery & Suspense', 'url': 'https://amorabooks.com/category/mystery-suspense'}, {'name': 'Sci-Fi & Fantasy Realms', 'url': 'https://amorabooks.com/category/scifi-fantasy-realms'}]


> Entering new CrewAgentExecutor chain...
First, I need to create four unique interlinking sentences that include semantically related category names and their respective URLs. To ensure quality and adherence to the guidelines, I will delegate this task to the Content Integration Manager. They have the necessary expertise to create high-quality content that meets the required criteria.

Action: Delegate work to coworker
A